In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

# Load Dataset

In [2]:
! ls ../Data

test_set_values.csv     training_set_labels.csv training_set_values.csv


Dataset comes split in 3 different files, training values, training labes and test values.
since we dont have a file for test-labels, We need to merge training values and training labes, re-split and we'll get test data needed to run a confusion matrix and get scores.

In [3]:
df_X = pd.read_csv('../data/training_set_values.csv')

In [4]:
df_X.shape

(59400, 40)

In [5]:
df_X.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [7]:
df_X.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

At first sight, we have Null values in some of the columns.

## Load training labels dataset


In [10]:
df_y = pd.read_csv('../data/training_set_labels.csv')

In [11]:
df_y.shape

(59400, 2)

In [12]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [13]:
df_y['status_group'].unique()

array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

Will have to work with a multiclass model, we have 3 different classes

In [14]:
df_y['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

We could replace 'functional needs repair'  with either of the other classes just try performance on a binary model

## Data exploration 

merging target and features into a single dataframe, just in case rows need to be dropped, and later spliting it 

In [15]:
df_raw = df_X.merge(df_y, on='id')

In [16]:
df_raw

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


## Meaning provided for each of the columns on [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/)

`amount_tsh` - Total static head (amount water available to waterpoint)

`date_recorded` - The date the row was entered

`funder` - Who funded the well

`gps_height` - Altitude of the well

`installer` - Organization that installed the well

`longitude` - GPS coordinate

`latitude` - GPS coordinate

`wpt_name` - Name of the waterpoint if there is one

`num_private` -

`basin` - Geographic water basin

`subvillage` - Geographic location

`region` - Geographic location

`region_code` - Geographic location (coded)

`district_code` - Geographic location (coded)

`lga` - Geographic location

`ward` - Geographic location

`population` - Population around the well

`public_meeting` - True/False

`recorded_by` - Group entering this row of data

`scheme_management` - Who operates the waterpoint

`scheme_name` - Who operates the waterpoint

`permit` - If the waterpoint is permitted

`construction_year` - Year the waterpoint was constructed

`extraction_type` - The kind of extraction the waterpoint uses

`extraction_type_group` - The kind of extraction the waterpoint uses

`extraction_type_class` - The kind of extraction the waterpoint uses

`management` - How the waterpoint is managed

`management_group` - How the waterpoint is managed

`payment` - What the water costs

`payment_type` - What the water costs

`water_quality` - The quality of the water

`quality_group` - The quality of the water

`quantity` - The quantity of water

`quantity_group` - The quantity of water

`source` - The source of the water

`source_type` - The source of the water

`source_class` - The source of the water

`waterpoint_type` - The kind of waterpoint

`waterpoint_type_group` - The kind of waterpoint

In [17]:
df_raw.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

By team decision, i will take care of following nulls, and find an apropiate way to proceed 

`subvillage`...................371 nulls

`public_meeting`..........3334 nulls


In [18]:
df_raw['public_meeting']

0        True
1         NaN
2        True
3        True
4        True
         ... 
59395    True
59396    True
59397    True
59398    True
59399    True
Name: public_meeting, Length: 59400, dtype: object

There's no information on what `public_meeting` column represents, unable to find any useful info on neither source page or after runing web search. Will possibly drop column

Looking into `subvillage` column, trying to find pattern and fill null values accordingly

In [19]:
df_subvillage_null = df_raw[df_raw['subvillage'].isnull()]

In [20]:
df_subvillage_null

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
342,21127,0.0,2011-03-16,Government Of Tanzania,0,North,36.557631,-6.233394,Kwa Mihinzo,0,...,soft,good,dry,dry,spring,spring,groundwater,communal standpipe multiple,communal standpipe,non functional
360,51558,0.0,2011-03-25,Commu,0,Commu,36.416701,-6.220157,Kwa Emanuel,0,...,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe,non functional
379,53847,0.0,2011-03-20,World Bank,0,Rhobi,36.729383,-6.084255,Kwa Dimanyi,0,...,salty,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional
565,27334,0.0,2011-03-18,World Bank,0,Rhoda,36.696881,-5.993192,Mpande,0,...,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
966,17088,0.0,2011-03-11,Water,0,Commu,36.322623,-6.030500,Kwa Charles,0,...,salty,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59008,16353,0.0,2011-03-23,Commu,0,Commu,36.770490,-6.288555,Kwa Mlima,0,...,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59091,45206,0.0,2011-03-14,Lvia,0,Commu,36.407974,-5.715084,Kwa Emson,0,...,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,non functional
59105,12248,0.0,2011-03-19,World Bank,0,Rhobi,36.889359,-5.959966,Kwa Mahimbo,0,...,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
59215,46441,0.0,2011-03-19,World Bank,0,Rhobi,36.854216,-6.010508,Kwa Sila,0,...,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional


Dataframe has 41 columns, it doesnt display all at once, will print not shown columns below, and take a look at them

In [21]:
df_subvillage_null.iloc[:,10:31]

,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,...,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type
342,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Sagara,0,True,GeoData Consultants Ltd,...,Saga,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay
360,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Kongwa Urban,0,True,GeoData Consultants Ltd,...,Mlan,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay
379,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Pandambili,0,True,GeoData Consultants Ltd,...,Pand,False,0,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket
565,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Njoge,0,True,GeoData Consultants Ltd,...,Hemb,False,0,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket
966,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Sejeli,0,True,GeoData Consultants Ltd,...,Seje,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59008,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Mlali,0,True,GeoData Consultants Ltd,...,Mlal,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay
59091,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Zoissa,0,True,GeoData Consultants Ltd,...,Zois,False,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket
59105,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Pandambili,0,True,GeoData Consultants Ltd,...,Kite,False,0,submersible,submersible,submersible,vwc,user-group,never pay,never pay
59215,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Pandambili,0,True,GeoData Consultants Ltd,...,Kite,False,0,submersible,submersible,submersible,vwc,user-group,never pay,never pay


Seems like most null values in the `subvillage` column are from region Dodoma, and could be posible not village but just wards near the capital, dodoma, that could explain the Null value

In [22]:
df_subvillage_null['region'].value_counts()

Dodoma    361
Mwanza     10
Name: region, dtype: int64

Out of all `subvillage` null values, 361 show Dodoma on `region`, other 10 show Mwanza.
All show the same region code, and district_code, it is safe to assume all null values represent a single value, maybe an area not identified as a village.

looks like `distric_code` is actually representing the region, http://www.statoids.com/utz.html 
we could drop one of those columns, or both in case the geographical data in not needed


`amount_tsh` shows lots of 0s, and in some data points the well doesnt have any water but the mecanism to make it work is still functional..?

In [23]:
df_subvillage_null['amount_tsh'].value_counts()

0.0    371
Name: amount_tsh, dtype: int64

In [24]:
df_raw['amount_tsh'].value_counts()

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

All rows where subvillage is null show a 0 on the `amount_tsh` column, but on the whole dataset it goes up to 41 thousand rows showing a 0 value,  we might have to drop it

#### Exploring other columns

There are some duplicate/redundant columns, will take a closer look at them

In [37]:
df_raw['num_private'].value_counts().head(10)



0     58643
6        81
1        73
5        46
8        46
32       40
45       36
15       35
39       30
93       28
Name: num_private, dtype: int64

Lots of 0 values. will drop the column

<br>

`water_quality` and `quality_group` columns, seem like containing similar values

In [40]:
print(df_raw['water_quality'].unique())

print(df_raw['quality_group'].unique())

['soft' 'salty' 'milky' 'unknown' 'fluoride' 'coloured' 'salty abandoned'
 'fluoride abandoned']
['good' 'salty' 'milky' 'unknown' 'fluoride' 'colored']


In [44]:
print(df_raw['water_quality'].value_counts())

print('-----------------------------')

print(df_raw['quality_group'].value_counts())

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64
-----------------------------
good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64


Both columns share similar values, `water_quality` have 2 more options fot cataloging the water quality, i will drop `water_quality` and keep `quality_group` since it groups 'fluoride' and 'fluoride abandoned' together and still have the smallest count our of all the category.


<br>

We have a similar situation with `quantity` and `quantity_group`, but in this case both are exacly the same, even on value counts

In [46]:
print(df_raw['quantity'].unique())

print(df_raw['quantity_group'].unique())

['enough' 'insufficient' 'dry' 'seasonal' 'unknown']
['enough' 'insufficient' 'dry' 'seasonal' 'unknown']


In [45]:
print(df_raw['quantity'].value_counts())

print('-----------------------------')

print(df_raw['quantity_group'].value_counts())

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64
-----------------------------
enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64


Will keep `quantity` and drop `quantity_group`

<br>

Let's take a look at `source` and `source_type`

In [47]:
print(df_raw['source'].unique())

print(df_raw['source_type'].unique())

['spring' 'rainwater harvesting' 'dam' 'machine dbh' 'other'
 'shallow well' 'river' 'hand dtw' 'lake' 'unknown']
['spring' 'rainwater harvesting' 'dam' 'borehole' 'other' 'shallow well'
 'river/lake']


In [48]:
print(df_raw['source'].value_counts())

print('-----------------------------')

print(df_raw['source_type'].value_counts())

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64
-----------------------------
spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64


Will keep `source_type` since it merges 'other' and 'unknown' together

<br>

`waterpoint_type` and `waterpoint_type_group`

In [51]:
print(df_raw['waterpoint_type'].unique())

print(df_raw['waterpoint_type_group'].unique())

['communal standpipe' 'communal standpipe multiple' 'hand pump' 'other'
 'improved spring' 'cattle trough' 'dam']
['communal standpipe' 'hand pump' 'other' 'improved spring'
 'cattle trough' 'dam']


In [52]:
print(df_raw['waterpoint_type'].value_counts())

print('-----------------------------')

print(df_raw['waterpoint_type_group'].value_counts())

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64
-----------------------------
communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64


`waterpoint_type_group` keeps 'communal standpipe multiple' and 'communal standpipe' values together, will keep that column

<br>

`extraction_type`, `extraction_type_group` and `extraction_type_class`

In [64]:
print(df_raw['extraction_type'].unique())

print(df_raw['extraction_type_group'].unique())

print(df_raw['extraction_type_class'].unique())

['gravity' 'submersible' 'swn 80' 'nira/tanira' 'india mark ii' 'other'
 'ksb' 'mono' 'windmill' 'afridev' 'other - rope pump' 'india mark iii'
 'other - swn 81' 'other - play pump' 'cemo' 'climax' 'walimi'
 'other - mkulima/shinyanga']
['gravity' 'submersible' 'swn 80' 'nira/tanira' 'india mark ii' 'other'
 'mono' 'wind-powered' 'afridev' 'rope pump' 'india mark iii'
 'other handpump' 'other motorpump']
['gravity' 'submersible' 'handpump' 'other' 'motorpump' 'wind-powered'
 'rope pump']


In [65]:
print(df_raw['extraction_type'].value_counts())

print('-----------------------------')

print(df_raw['extraction_type_group'].value_counts())

print('-----------------------------')

print(df_raw['extraction_type_class'].value_counts())

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64
-----------------------------
gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
indi

`extraction_type` and `extraction_type_group` seems semilar and if we compare them to `extraction_type_class`, the first two have the specific kind of machine/mechanic used to extract the water, whereas `extraction_type_class` is summarizing all of those as 'handpump'.

Will Keep `extraction_type_group` and `extraction_type_class` since there could be a relation between the kind of machine and its status.

<br>

`payment` and `payment_type`

In [66]:
print(df_raw['payment'].unique())

print(df_raw['payment_type'].unique())

['pay annually' 'never pay' 'pay per bucket' 'unknown'
 'pay when scheme fails' 'other' 'pay monthly']
['annually' 'never pay' 'per bucket' 'unknown' 'on failure' 'other'
 'monthly']


In [67]:
print(df_raw['payment'].value_counts())

print('-----------------------------')

print(df_raw['payment_type'].value_counts())

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64
-----------------------------
never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64


They look the same, will keep `payment`

<br>

`construction_year`

In [68]:
df_raw['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

Over 20 thousand missing values, will drop column. Same situation with `population`.

In [69]:
df_raw['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

### Droping columns

Will also drop any column including names or location information, with the exception of `ward` and `region`, will see how model performs with those values.

In [70]:
df_raw.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [82]:
df_Dropped = df_raw.drop(columns=['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height', 
                                  'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 
                                  'subvillage', 'region_code', 'district_code', 'lga', 
                                  'population', 'public_meeting', 'recorded_by', 'scheme_management', 
                                  'scheme_name', 'construction_year', 'extraction_type', 'management', 
                                  'management_group', 'payment_type', 'water_quality', 'quantity_group', 
                                  'source', 'waterpoint_type', 'permit'])

df_Dropped

,basin,region,ward,extraction_type_group,extraction_type_class,payment,quality_group,quantity,source_type,source_class,waterpoint_type_group,status_group
0,Lake Nyasa,Iringa,Mundindi,gravity,gravity,pay annually,good,enough,spring,groundwater,communal standpipe,functional
1,Lake Victoria,Mara,Natta,gravity,gravity,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional
2,Pangani,Manyara,Ngorika,gravity,gravity,pay per bucket,good,enough,dam,surface,communal standpipe,functional
3,Ruvuma / Southern Coast,Mtwara,Nanyumbu,submersible,submersible,never pay,good,dry,borehole,groundwater,communal standpipe,non functional
4,Lake Victoria,Kagera,Nyakasimbi,gravity,gravity,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...
59395,Pangani,Kilimanjaro,Masama Magharibi,gravity,gravity,pay per bucket,good,enough,spring,groundwater,communal standpipe,functional
59396,Rufiji,Iringa,Ikondo,gravity,gravity,pay annually,good,enough,river/lake,surface,communal standpipe,functional
59397,Rufiji,Mbeya,Chimala,swn 80,handpump,pay monthly,fluoride,enough,borehole,groundwater,hand pump,functional
59398,Rufiji,Dodoma,Mvumi Makulu,nira/tanira,handpump,never pay,good,insufficient,shallow well,groundwater,hand pump,functional


In [83]:
df_Dropped.shape

(59400, 12)

# Spliting data

In [84]:
X = df_Dropped.drop(columns='status_group')
y = df_Dropped['status_group']

X_train, X_test, y_train, y_test  =  train_test_split(X, y,  random_state=42)

# Simple models

#### Logistic regression

Will try running a binary model, our target has 3 clases, will replace 'functional needs repair' to 'non functional'

In [99]:
y_train_binary = y_train.map({'functional':'functional', 'non functional':'non functional',
                              'functional needs repair':'non functional'})
y_train_binary.value_counts()

functional                 24161
non functional             17146
functional needs repair     3243
Name: status_group, dtype: int64

In [103]:
logre = LogisticRegression(random_state=42, )

X_dumie_train = pd.get_dummies(X_train, drop_first=True)

logre.fit(X_dumie_train, y_train_binary, )

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [104]:
logre.score(X_dumie_train, y_train_binary)

0.7984736251402919

Transforming tests values, to get score

In [ ]:
logre.score(X_dumie_train, y_train_binary)

# not used code

## will create a column showing if the record was taken on dry month or other, using the date_recorder column


https://www.nathab.com/know-before-you-go/african-safaris/east-africa/weather-climate/tanzania/#:~:text=The%20dry%20season%2C%20with%20cooler,a%20chance%20of%20afternoon%20showers.


The dry season, with cooler temperatures, lasts from May to October.

Summer usually lasts from November to March, and during this time there is always a chance of afternoon showers. 

and april shows the highes average of inches of rain per month

jan is considered a short dry season between, months that rains alot.




df_Dropped['date_recorded'][1][5:7]

https://altezza.travel/en/articles/weather

df_Dropped['season'] = pd.DatetimeIndex(df_Dropped['date_recorded']).month

df_Dropped ['precipitation (mm)'] = df_Dropped['season'].map({1:49, 2:30, 3:85, 4:153, 5:126, 6:32,
                                                         7:13, 8:18, 9:21, 10:48, 11:132, 12:75})
df_Dropped.head ( )

df_Dropped.drop(columns=['season', 'date_recorded'], inplace=True)

df_Dropped

## creating a simple model 

X = df_Dropped.drop(columns=['status_group', 'management', 'management_group', 
                             'scheme_management', 'scheme_name', 'permit'])
y = df_Dropped['status_group']

X_dummie = pd.get_dummies(data= X,  drop_first=True)

X_dummie

tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_clf.fit(X_dummie, y)

tree_clf.score(X_dummie, y)

LOL